This program give information on how much one will need to retire.

In [1]:
import pandas as pd
import numpy as np
import time

In [10]:
###################################################
# METHODS
###################################################

def init_df():
    df = pd.DataFrame(columns= ['APR', 'Year Number', 'Starting Balance', 
                                'Ending Balance', 'Additional Premium'])
    return df

def impact_years_compounding(df, starting_principle, yrs, apr, additional_premium, 
                             years_to_retire, how_much_needed_at_retirement):
    yr = 1
    while yr <= yrs: #calculates compounding across multiple years
        
        if yr < years_to_retire:
            debit = 0
        else:
            debit = -how_much_needed_at_retirement
            
        final_principle, apr = impact_range_of_annual_percentage_rates(starting_principle, 
                                                                       apr, additional_premium, 
                                                                       debit)
        
        
        
        df = df.append({'APR': apr, 'Year Number': yr, 'Starting Balance': starting_principle, 
                        'Ending Balance': final_principle, 'Additional Premium' : additional_premium, 
                        'Debit' : debit} , ignore_index = True)
        
        starting_principle = final_principle
        
        yr += 1
        
    return df
    

def impact_range_of_annual_percentage_rates(starting_principle, apr, additional_premium, 
                                            debit): #
        
        final_principle, apr = annual_by_month_compound(starting_principle , apr, 
                                                        additional_premium, debit) #ending principle after 1 year
        '''print('Your starting principle was: ', how_much_saved)
        print('The annual interest rate (APR) is: ', apr, '%')
        print('The annualized final principle amortized monthly is: ', final_principle, '\n')
        '''
        return final_principle , apr #output is the final principle from apr/12 interest compounded montly for 1 year
        
def annual_by_month_compound(how_much_saved, apr, additional_principle, debit): #interest rate compounded monthly for one year
    mpr = apr/1200 # APR divided by 12 to get monthly interest rate
    #print('mpr = ', mpr)
    month = 1    
    new_principle = float(how_much_saved)
    #print('new_principle = ', new_principle, '\n')
    while month < 13:
        gain_from_interest_on_principle = new_principle * mpr
        #print('gain_from_interest_on_principle =', gain_from_interest_on_principle)
        new_principle = new_principle + gain_from_interest_on_principle + additional_principle/12 + debit/12
        month += 1
    return new_principle, apr



In [16]:
#########################################
# MAIN #
#########################################

# Variable definition

years_to_retire = int(input('In how many years do you plan to retire? '))
how_much_needed_at_retirement = float(input('How much per year will you need at retirement? '))
starting_principle = float(input('How much have you saved for retirement right now? [in thousands]'))
how_much_ss_at_retirement = input('How much Social Security benefits will you get at retirement? ')
how_much_other_passive_income = float(input('How much other passive income will you get at retirement? '))
apr = float(input('What is the annualized percent return? '))
yrs = int(input('How long will you be getting this interest? '))
additional_premium = float(input('How much will you be adding to your retirement savings per year? '))

df = init_df()

df = impact_years_compounding(df, starting_principle, yrs, apr, additional_premium, 
                             years_to_retire, how_much_needed_at_retirement)

print(df)



In how many years do you plan to retire? 5
How much per year will you need at retirement? 50
How much have you saved for retirement right now? [in thousands]200
How much Social Security benefits will you get at retirement? 15
How much other passive income will you get at retirement? 10
What is the annualized percent return? 15
How long will you be getting this interest? 15
How much will you be adding to your retirement savings per year? 10
     APR  Year Number  Starting Balance  Ending Balance  Additional Premium  \
0   15.0          1.0        200.000000      242.867871                10.0   
1   15.0          2.0        242.867871      292.626947                10.0   
2   15.0          3.0        292.626947      350.385018                10.0   
3   15.0          4.0        350.385018      417.427961                10.0   
4   15.0          5.0        417.427961      441.663520                10.0   
5   15.0          6.0        441.663520      469.795055                10.0   
6  

In [ ]:
print(df)

In [ ]:
df = pd.DataFrame(columns= ['APR', 'Years', 'Starting Balance', 'Ending Balance'])
list(df.columns)

In [ ]:
i = 0

while i < 5:
        
    APR = i
    Years = i
    S = i
    B = i 
    
    df = df.append({'APR': APR, 'Years': Years, 'Starting Balance': S, 'Ending Balance': B} , ignore_index = True)

    i += 1
        
print(df)

In [ ]:
print(df)

In [ ]:
df = pd.DataFrame(columns=['A'])
for i in range(5):
    df = df.append({'A': i}, ignore_index=True)
df